# Main Notebook acting as a User Interface

## ShopAssist AI: Intelligent Laptop Shopping Assistant

**Project Overview**
- ShopAssist AI is an innovative chatbot designed to transform the online laptop shopping experience. By leveraging the power of Large Language Models (LLMs) and rule-based functions, this intelligent assistant provides accurate and personalized laptop recommendations based on user requirements. ShopAssist AI aims to simplify the overwhelming nature of online shopping by guiding users in finding the perfect laptop tailored to their needs.

**Problem Statement**
- In the digital shopping era, the abundance of options can make it difficult for consumers to make informed decisions. This project addresses the challenge of:

    - Interacting with users to gather their requirements.
    - Parsing a dataset of laptops to extract relevant information.
    - Recommending the most suitable laptops based on user preferences.
    - 
**Key Objectives**:
- Engage users: Provide natural and interactive conversations to understand user requirements.
- Extract insights: Use LLMs and rule-based reasoning to map user preferences to the dataset.
- Recommend effectively: Suggest the top three laptops matching user needs and answer follow-up queries.

**Solution Approach**
1. Conversation and Information Gathering
- The chatbot initiates a natural conversation to collect user requirements, such as budget, specifications, and usage needs.
Moderation checks ensure the conversation is safe and free of sensitive content.
2. Information Extraction
- The system analyzes user requirements using natural language understanding (NLU).
It filters laptops from the dataset based on extracted features and calculates compatibility scores.
3. Personalized Recommendations
- Presents a maximum of three laptops, ranked by relevance to user requirements.
In cases where no laptops meet the threshold, the system connects the user with a human expert.


### Stage 1: Intent Clarity and Confirmation
This stage involves initiating and managing the conversation between the user and the AI system.

Key Functions:
- initialize_conversation(): Starts the conversation with the user.
- get_chat_completions(): Continuously processes user inputs and generates LLM responses.
- moderation_check(): Flags and halts conversations containing unsafe or sensitive content.

### Stage 2: Product Mapping and Information Extraction
This stage filters laptops based on user requirements and identifies the top recommendations.

Key Functions:
- product_map_layer(): Extracts and maps key features (e.g., GPU intensity, display quality) from the dataset.
- compare_laptops_with_user(): Matches user requirements against the laptop features to calculate scores.
- recommendation_validation(): Validates the recommendations to ensure they meet quality thresholds.

### Stage 3: Product Recommendation
This stage delivers recommendations and engages in further conversation.

Key Functions:
- initialize_conv_reco(): Generates a structured conversation with summarized laptop recommendations.
- get_chat_completions(): Facilitates follow-up queries and detailed discussions about the recommended laptops.

![complete_flow.jpg](/Users/imkushwaha/IIITB-AIML/ShopAssistApp/app/img/CompleteFlow.png)

In [1]:
import os
import openai
from shopassist.services.stage1 import (initialize_conversation, 
                                        get_chat_completions, 
                                        moderation_check, 
                                        intent_confirmation_layer,
                                        dictionary_present)
from shopassist.services.stage3 import initialize_conv_reco
from shopassist.services.stage2 import (compare_laptops_with_user, 
                                        recommendation_validation)

In [4]:
OPENAI_API_KEY_PATH = os.path.join("shopassist", "config", "OPENAI_API_KEY.txt")
openai.api_key = open(OPENAI_API_KEY_PATH, "r").read().strip()
os.environ['OPENAI_API_KEY'] = openai.api_key

In [ ]:
def dialogue_mgmt_system():
    conversation = initialize_conversation()

    introduction = get_chat_completions(conversation)

    display(introduction + '\n')

    top_3_laptops = None

    user_input = ''

    while(user_input != "exit"):

        user_input = input("")

        moderation = moderation_check(user_input)
        if moderation == 'Flagged':
            display("Sorry, this message has been flagged. Please restart your conversation.")
            break

        if top_3_laptops is None:

            conversation.append({"role": "user", "content": user_input})

            response_assistant = get_chat_completions(conversation)
            moderation = moderation_check(response_assistant)
            if moderation == 'Flagged':
                display("Sorry, this message has been flagged. Please restart your conversation.")
                break


            confirmation = intent_confirmation_layer(response_assistant)

            print("Intent Confirmation Yes/No:",confirmation.get('result'))

            if "No" in confirmation.get('result'):
                conversation.append({"role": "assistant", "content": str(response_assistant)})
                print("\n" + str(response_assistant) + "\n")

            else:
                print("\n" + str(response_assistant) + "\n")
                print('\n' + "Variables extracted!" + '\n')

                response = dictionary_present(response_assistant)

                print("Thank you for providing all the information. Kindly wait, while I fetch the products: \n")
                top_3_laptops = compare_laptops_with_user(response)

                print("top 3 laptops are", top_3_laptops)

                validated_reco = recommendation_validation(top_3_laptops)

                conversation_reco = initialize_conv_reco(validated_reco)

                conversation_reco.append({"role": "user", "content": "This is my user profile" + str(response)})

                recommendation = get_chat_completions(conversation_reco)

                moderation = moderation_check(recommendation)
                if moderation == 'Flagged':
                    display("Sorry, this message has been flagged. Please restart your conversation.")
                    break

                conversation_reco.append({"role": "assistant", "content": str(recommendation)})

                print(str(recommendation) + '\n')
        else:
            conversation_reco.append({"role": "user", "content": user_input})

            response_asst_reco = get_chat_completions(conversation_reco)

            moderation = moderation_check(response_asst_reco)
            if moderation == 'Flagged':
                print("Sorry, this message has been flagged. Please restart your conversation.")
                break

            print('\n' + response_asst_reco + '\n')
            conversation.append({"role": "assistant", "content": response_asst_reco})